In [2]:
import cntk as C
import cntk.tests.test_utils
import cntk_unet
from cntk.learners import learning_rate_schedule, UnitType
from cntk.device import try_set_default_device, gpu, all_devices
print(all_devices())
try_set_default_device(gpu(0))
cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
C.cntk_py.set_fixed_random_seed(1) # fix the random seed so that LR examples are repeatable

import numpy as np
import sys
import os
import mathfrom datetime import datetime
from pytz import timezone
from matplotlib import pyplot as plt
from collections import defaultdict
from cntk.initializer import normal

(GPU[0] Tesla M60, CPU)


In [3]:
import numpy as np
import sys
import os
import math
import cntk
from datetime import datetime
from pytz import timezone
from matplotlib import pyplot as plt
from collections import defaultdict
from cntk.initializer import normal
global f 
global index_model
index_model = 1
model_list = []


In [4]:
import cntk as C
from cntk.layers import Convolution, MaxPooling, Dense
from cntk.initializer import glorot_uniform
from cntk.ops import relu, sigmoid, input_variable

def UpSampling2D(x):
    xr = C.reshape(x, (x.shape[0], x.shape[1], 1, x.shape[2], 1))
    xx = C.splice(xr, xr, axis=-1) # axis=-1 refers to the last axis
    xy = C.splice(xx, xx, axis=-3) # axis=-3 refers to the middle axis
    r = C.reshape(xy, (x.shape[0], x.shape[1] * 2, x.shape[2] * 2))

    return r

def create_model(input, num_classes):
    conv1 = Convolution((3,3), 32, init=glorot_uniform(), activation=relu, pad=True)(input)
    conv1 = Convolution((3,3), 32, init=glorot_uniform(), activation=relu, pad=True)(conv1)
    pool1 = MaxPooling((2,2), strides=(2,2))(conv1)

    conv2 = Convolution((3,3), 64, init=glorot_uniform(), activation=relu, pad=True)(pool1)
    conv2 = Convolution((3,3), 64, init=glorot_uniform(), activation=relu, pad=True)(conv2)
    pool2 = MaxPooling((2,2), strides=(2,2))(conv2)

    conv3 = Convolution((3,3), 128, init=glorot_uniform(), activation=relu, pad=True)(pool2)
    conv3 = Convolution((3,3), 128, init=glorot_uniform(), activation=relu, pad=True)(conv3)
    pool3 = MaxPooling((2,2), strides=(2,2))(conv3)

    conv4 = Convolution((3,3), 256, init=glorot_uniform(), activation=relu, pad=True)(pool3)
    conv4 = Convolution((3,3), 256, init=glorot_uniform(), activation=relu, pad=True)(conv4)
    pool4 = MaxPooling((2,2), strides=(2,2))(conv4)

    conv5 = Convolution((3,3), 512, init=glorot_uniform(), activation=relu, pad=True)(pool4)
    conv5 = Convolution((3,3), 512, init=glorot_uniform(), activation=relu, pad=True)(conv5)
   
    up6 = C.splice(UpSampling2D(conv5), conv4, axis=0)
    conv6 = Convolution((3,3), 256, init=glorot_uniform(), activation=relu, pad=True)(up6)
    conv6 = Convolution((3,3), 256, init=glorot_uniform(), activation=relu, pad=True)(conv6)

    up7 = C.splice(UpSampling2D(conv6), conv3, axis=0)
    conv7 = Convolution((3,3), 128, init=glorot_uniform(), activation=relu, pad=True)(up7)
    conv7 = Convolution((3,3), 128, init=glorot_uniform(), activation=relu, pad=True)(conv7)

    up8 = C.splice(UpSampling2D(conv7), conv2, axis=0)
    conv8 = Convolution((3,3), 64, init=glorot_uniform(), activation=relu, pad=True)(up8)
    conv8 = Convolution((3,3), 64, init=glorot_uniform(), activation=relu, pad=True)(conv8)

    up9 = C.splice(UpSampling2D(conv8), conv1, axis=0)
    conv9 = Convolution((3,3), 64, init=glorot_uniform(), activation=relu, pad=True)(up9)
    conv9 = Convolution((3,3), 64, init=glorot_uniform(), activation=relu, pad=True)(conv9)

    conv10 = Convolution((1,1), num_classes, init=glorot_uniform(), activation=sigmoid, pad=True)(conv9)

    return conv10

def dice_coefficient(x, y):
    # average of per-channel dice coefficient
    # global dice coefificnet doesn't work as class with larger region dominates the metrics
    # https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
    intersection = C.reduce_sum(x * y, axis=(1,2))
    dice = C.reduce_mean(2.0 * intersection / (C.reduce_sum(x, axis=(1,2)) + C.reduce_sum(y, axis=(1,2)) + 1.0))
    print("dice = " + str(dice))

    return dice

In [5]:
def slice_minibatch(all_features, all_labels, i, minibatch_size):
    batch_features = all_features[i * minibatch_size:(i + 1) * minibatch_size]
    batch_labels = all_labels[i * minibatch_size:(i + 1) * minibatch_size]

    return batch_features, batch_labels

def measure_error(data_x, data_y, x, y, trainer, minibatch_size):
    errors = []
    for i in range(0, int(len(data_x) / minibatch_size)):
        data_sx, data_sy = slice_minibatch(data_x, data_y, i, minibatch_size)

        errors.append(trainer.test_minibatch({x: data_sx, y: data_sy}))

    return np.mean(errors)

def train(images, masks, dst_folder, file_writer, use_existing=False):
    shape = images[0].shape
    data_size = images.shape[0]
    print(shape)
    print(data_size)

    # Split data
    test_portion = int(data_size * 0.1)
    indices = np.random.permutation(data_size)
    test_indices = indices[:test_portion]
    training_indices = indices[test_portion:]

    test_data = (images[test_indices], masks[test_indices])
    training_data = (images[training_indices], masks[training_indices])

    # Create model
    x = C.input_variable(shape)
    y = C.input_variable(masks[0].shape)

    z = create_model(x, masks.shape[1])
    dice_coef = dice_coefficient(z, y)

    # Load the saved model if specified
    checkpoint_file = "cntk-unet.dnn"
    if use_existing:
        z.load_model(checkpoint_file)

    # Prepare model and trainer
    lr = learning_rate_schedule(0.00001, UnitType.sample)
    momentum = C.learners.momentum_as_time_constant_schedule(0.9)
    trainer = C.Trainer(z, (-dice_coef, -dice_coef), C.learners.adam(z.parameters, lr=lr, momentum=momentum))

    # Get minibatches of training data and perform model training
    minibatch_size = 8
    num_epochs = 75

    training_errors = []
    test_errors = []

    for e in range(0, num_epochs):
        for i in range(0, int(len(training_data[0]) / minibatch_size)):
            batch_features, batch_labels = slice_minibatch(training_data[0], training_data[1], i, minibatch_size)

            trainer.train_minibatch({x:batch_features, y: batch_labels})

        # Measure training error
        training_error = measure_error(training_data[0], training_data[1], x, y, trainer, minibatch_size)
        training_errors.append(training_error)

        # Measure test error
        test_error = measure_error(test_data[0], test_data[1], x, y, trainer, minibatch_size)
        test_errors.append(test_error)

        string_log = "epoch #{}: training_error={}, test_error={}".format(e, training_errors[-1], test_errors[-1])
        string_append_and_print(string_log, file_writer)
        
#         string_saved_model = "\t\tmodel saved from data set {0:.2f}".format(test_error*100 / num_minibatches_to_test)
#         string_append_and_print(string_saved_model, path_log)
        
        trainer.save_checkpoint(checkpoint_file)
#         z.save(dst_folder + "/model{0:.2f}.model".format(test_result*100 / num_minibatches_to_test))
    return trainer, training_errors, test_errors

In [6]:
import matplotlib.pyplot as plt
import numpy as np

def plot_img_array(img_array, ncol=3):
    nrow = len(img_array) // ncol

    f, plots = plt.subplots(nrow, ncol, sharex='all', sharey='all', figsize=(ncol * 4, nrow * 4))

    for i in range(len(img_array)):
        plots[i // ncol, i % ncol]
        plots[i // ncol, i % ncol].imshow(img_array[i])

from functools import reduce
def plot_side_by_side(img_arrays):
    flatten_list = reduce(lambda x,y: x+y, zip(*img_arrays))

    plot_img_array(np.array(flatten_list), ncol=len(img_arrays))

import itertools
def plot_errors(results_dict, title):
    markers = itertools.cycle(('+', 'x', 'o'))

    plt.title('{}'.format(title))

    for label, result in sorted(results_dict.items()):
        plt.plot(result, marker=next(markers), label=label)
        plt.ylabel('dice_coef')
        plt.xlabel('epoch')
        plt.legend(loc=3, bbox_to_anchor=(1, 0))

    plt.show()

def masks_to_colorimg(masks):
    colors = np.asarray([(201, 58, 64), (242, 207, 1), (0, 152, 75), (101, 172, 228),(56, 34, 132), (160, 194, 56)])

    colorimg = np.ones((masks.shape[1], masks.shape[2], 3), dtype=np.float32) * 255
    channels, height, width = masks.shape

    for y in range(height):
        for x in range(width):
            selected_colors = colors[masks[:,y,x] > 0.5]

            if len(selected_colors) > 0:
                colorimg[y,x,:] = np.mean(selected_colors, axis=0)

    return colorimg.astype(np.uint8)

In [7]:
def create_reader_text(path, is_training, input_dim, num_label_classes):   
    print(path)
    labelStream = cntk.io.StreamDef(field='label', shape=(256, 256), is_sparse=False)
    featureStream = cntk.io.StreamDef(field='features', shape=(3, 256, 256), is_sparse=False)
#     coordinateStream = cntk.io.StreamDef(field='coordinate', shape=2, is_sparse=False)
    
#     deserailizer = cntk.io.CBFDeserializer(path, cntk.io.StreamDefs(label = labelStream ,feature = featureStream))
    deserailizer = cntk.io.CTFDeserializer(path, cntk.io.StreamDefs(label = labelStream ,feature = featureStream))      
    return cntk.io.MinibatchSource(deserailizer,
       randomize = False, multithreaded_deserializer=False,max_sweeps = cntk.io.INFINITELY_REPEAT if is_training else 1)

def create_reader_binary(path, is_training, input_dim, num_label_classes):   
#     print(path)
    labelStream = cntk.io.StreamDef(field='label', shape=num_label_classes, is_sparse=False)
    featureStream = cntk.io.StreamDef(field='features', shape=input_dim, is_sparse=False)
#     coordinateStream = cntk.io.StreamDef(field='coordinate', shape=2, is_sparse=False)
    
    deserailizer = cntk.io.CBFDeserializer(path, cntk.io.StreamDefs(label = labelStream ,feature = featureStream))
#     deserailizer = cntk.io.CTFDeserializer(path, cntk.io.StreamDefs(label = labelStream ,feature = featureStream))      
    return cntk.io.MinibatchSource(deserailizer,
       randomize = False, multithreaded_deserializer=False,max_sweeps = cntk.io.INFINITELY_REPEAT if is_training else 1)

In [8]:
def create_criterion_function(model, labels):
    loss = cntk.cross_entropy_with_softmax(model, labels)
    errs = cntk.classification_error(model, labels)
    return loss, errs # (model, labels) -> (loss, error metric)

In [9]:
# Define a utility function to compute the moving average sum.
# A more efficient implementation is possible with np.cumsum() function
def moving_average(a, w=5):
    if len(a) < w:
        return a[:]    # Need to send a copy of the array
    return [val if idx < w else sum(a[(idx-w):idx])/w for idx, val in enumerate(a)]


# Defines a utility that prints the training progress
def print_training_progress(trainer, mb, frequency, verbose=1):
    training_loss = "NA"
    eval_error = "NA"

#     if mb%frequency == 0:
    training_loss = trainer.previous_minibatch_loss_average
    eval_error = trainer.previous_minibatch_evaluation_average
#         if verbose: 
    string_timing = "\t\tMinibatch: {0}, Loss: {1:.4f}, Error: {2:.2f}%".format(mb, training_loss, eval_error*100)
    string_append_and_print(string_timing, path_log)
        
    return mb, training_loss, eval_error

In [10]:
def set_adaptive_learner(model, base_lr_per_sample, minibatch_size, num_samples_per_sweep):
    lr_schedule = get_adaptive_learning_schedule(base_lr_per_sample, minibatch_size, num_samples_per_sweep)
    #momentum_time_constant = -minibatch_size / np.log(0.98)
    #momentum_time_constant = [3000]
    l2_reg_weight = 0.0001  # 0.0001
    #mm_schedule = cntk.momentum_as_time_constant_schedule(momentum_time_constant)
    mm_schedule = cntk.momentum_schedule(0.90)

    learner = cntk.momentum_sgd(model.parameters, lr_schedule, mm_schedule,
                           l2_regularization_weight=l2_reg_weight, unit_gain=True)

    return learner
    
def get_adaptive_learning_schedule(base_lr_per_sample, minibatch_size, num_samples_per_sweep):
    return cntk.learners.learning_parameter_schedule(base_lr_per_sample, minibatch_size, num_samples_per_sweep)


In [11]:
def string_append_and_print(line, file) :
    file_writer = open(file, 'a')
    file_writer.write(line + "\n")
    file_writer.close()
    print(line)

In [12]:
import math

In [13]:
def plot(plotdata):
    plt.plot(plotdata["mb"], plotdata["loss"], '--r')
    plt.xlabel('Minibatch number')
    plt.ylabel('Loss')
    plt.show()

In [14]:
input_dim_model = (3, 256, 256)
input_dim = 3 * 256 * 256
num_output_classes = 256 * 256
input = cntk.input_variable(input_dim_model)  # สังเกตว่าเราใช้ input_dim_model เป็นพารามิเตอร์แทนการใช้ input_dim
label = cntk.input_variable((256,256))

# samples_test_size = len(open("../training_data/features/original/samples_test.txt", "r").readlines())
samples_train_size = len(open("../training_data/features/original/samples_train.txt", "r").readlines())

In [15]:
import time
def train_test(train_reader, test_reader, model_func ,path_log, num_sweeps_to_train_with, num_samples_train, num_samples_test, dst) :
    model = model_func(input/255)
    
    # Instantiate the loss and error function
    loss, label_error = create_criterion_function(model, label)
    
    minibatch_size = 64

    num_samples_per_sweep = num_samples_train

    num_minibatches_to_train = (num_samples_per_sweep * num_sweeps_to_train_with) / minibatch_size
    
    # Instantiate the trainer object to drive the model training
    k = int(num_samples_per_sweep / minibatch_size)
    num_k = num_sweeps_to_train_with // 5
    learning_rate = ([0.2]*(k*num_k))+([0.1]*(k*num_k))+([0.05]*(k*num_k))+([0.025]*(k*num_k))+([0.0125]*(k*num_k))
#     learning_rate = [0.2] * k
    lr_schedule = cntk.learning_rate_schedule(learning_rate, cntk.UnitType.minibatch)
    learner =  set_adaptive_learner(z, learning_rate, minibatch_size, num_samples_per_sweep)
#     learner = cntk.sgd(z.parameters, lr_schedule)
#     dice_coef = dice_coefficient(z, label)
    trainer = cntk.Trainer(z, (loss, label_error), [learner])
    
    input_map={
        label  : train_reader.streams.label,
        input  : train_reader.streams.feature
    }  
    
    # Uncomment below for more detailed logging
    training_progress_output_freq = 0
     
    # Start a timer
    start = time.time()
    l_list = []
    error = []
    loss_per_sweep = []
    for i in range(0, int(num_minibatches_to_train)):

        # Read a mini batch from the training data file
        data = train_reader.next_minibatch(minibatch_size, input_map=input_map) 
        
        training_data = []
        for k in data.keys() :
            for v in data[k].as_sequences()[0] :
                 training_data.append(v)

        data_x, data_y = training_data
        data_x = np.reshape(data_x,(256,256))
        data_y = np.reshape(data_y,(3,256,256))

#         data_x, data_y 

        trainer.train_minibatch({input: data_y, label: data_x})

#         a, l, e = print_training_progress(trainer, i, training_progress_output_freq, verbose=1)
#         a, l, e = print_training_progress(trainer, i, training_progress_output_freq, verbose=1)
        if i % training_progress_output_freq == 0 :
# #             print(i)
            a, l, e = print_training_progress(trainer, i, training_progress_output_freq, verbose=1)
            l_list.append(l)
            error.append(e)
        if i % k == 0 :
            
            a, l, e = print_training_progress(trainer, i, training_progress_output_freq, verbose=1)
            string_training_time = "\t\ttraining took per sweep round = {:.0f} => {:.1f} sec".format((i+1)/k,time.time() - start)
            string_append_and_print(string_training_time, path_log)
            plotdata["mb"].append(i)
            plotdata["loss"].append(l)
            loss_per_sweep.append(l)
#             print("loss per sweep to train = " + str(l))
       
     
    # Print training time
    string_training_time = "\t\tTraining took {:.1f} sec".format(time.time() - start)
    string_append_and_print(string_training_time, path_log)
    
    # Test the model
    test_input_map = {
        label  : test_reader.streams.label,
        input  : test_reader.streams.feature
    }

    # Test data for trained model
    test_minibatch_size = 64
    
#     num_samples = len( open(name,'r').readlines() )
    num_samples = num_samples_test
    num_minibatches_to_test = num_samples_test // test_minibatch_size
    
    test_result = 0.0   

    for i in range(num_minibatches_to_test):
    
        # We are loading test data in batches specified by test_minibatch_size
        # Each data point in the minibatch is a MNIST digit image of 784 dimensions 
        # with one pixel per dimension that we will encode / decode with the 
        # trained model.
        data = test_reader.next_minibatch(test_minibatch_size, input_map=test_input_map)
        
       
        # reshpae data this here.
        eval_error = trainer.test_minibatch(data)
        test_result = test_result + eval_error
    
    # Average of evaluation errors of all test minibatches
    string_average_test_error = "\t\tAverage test error: {0:.2f}%".format(test_result*100 / num_minibatches_to_test)
    string_append_and_print(string_average_test_error, path_log)
  
    #save model whem error late less number target
    path_model = dst + "/model{0:.2f}.model".format(test_result*100 / num_minibatches_to_test)
#     if((test_result*100 / num_minibatches_to_test) < 9) :
    string_saved_model = "\t\tmodel saved from data set {0:.2f}".format(test_result*100 / num_minibatches_to_test)
    string_append_and_print(string_saved_model, path_log)
    model_list.append(z)
#     if (test_result*100 / num_minibatches_to_test) <= 8.00 :
    z.save(path_model)
    return "{0:.2f}".format(test_result*100 / num_minibatches_to_test), l_list, loss_per_sweep,error

In [16]:
def get_value(data):
    training_data = []
    for k in data.keys() :
        for v in data[k].as_sequences()[0] :
             training_data.append(v)

def reshape(data, shape) :
    return np.reshape(data, shape)

# def prepare_data(reader_train, reader_test) :
#     input_map={
#         label  : train_reader.streams.label,
#         input  : train_reader.streams.feature
#     } 
#     reader_train.next_minibatch(minibatch_size, input_map=input_map) 
    
        
#         data_x, data_y = training_data
#         data_x = np.reshape(data_x,(256,256))
#         data_y = np.reshape(data_y,(3,256,256))
        
#     test_input_map = {
#         label  : test_reader.streams.label,
#         input  : test_reader.streams.feature
#     }
    

def do_train_test(reader_train, reader_test, input_dim, path_log ,num_output_classes, num_sweeps_to_train_with, num_samples_train, num_samples_test, dst):
    global z
    z = create_model(input, 256*256)
    
#     return train(reader_train, reader_test, dst, path_log)
    return train_test(reader_train, reader_test, z, path_log, num_sweeps_to_train_with, num_samples_train, num_samples_test,dst)

In [17]:


def load_test(src) :
    src_test_file = src + "samples_test.dat"
    num_samples = samples_test_size
    
    return create_reader_binary(src_test_file, False, input_dim, num_output_classes), num_samples

def load_train(src) :
    src_train_file = src + "samples_train.dat"
    num_samples = samples_train_size
        
    return create_reader_binary(src_train_file, True, input_dim, num_output_classes), num_samples
    
def load_config(src) :
    train_per_sweep = 10
    reader_train, num_samples_train = load_train(src)
    reader_test, num_samples_test = load_test(src)
    
    return train_per_sweep, reader_train, reader_test, num_samples_train, num_samples_test

In [18]:
def check_folder_exist(src_folder) :
    return os.path.exists(src_folder)

def create_folder(src_folder) :
    folders = src_folder.split("/")
    current_folder = ""
    
    for folder in folders :
        current_folder += folder + "/"
        if not (folder == '' or folder == '.' or folder == '..') and not (check_folder_exist(current_folder)):
            os.mkdir(current_folder)
            

In [19]:
from datetime import datetime
def process(src_folder, dst_folder) :
    
    create_folder(dst_folder)
    time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')
    
    path_log = dst_folder + time + ".txt"
    file_writer = open(path_log, 'w')
    
    train_per_sweep, reader_train, reader_test, num_samples_train, num_samples_test = load_config(src_folder)
    string_filter_details = "folder => {:s}, train_per_sweep => {:d}\n".format(src_folder, train_per_sweep)
    
    
#     return do_train_test(reader_train, reader_test, input_dim, path_log, num_output_classes, train_per_sweep, num_samples_train, num_samples_test, dst_folder)
    return reader_train, reader_test

In [20]:
src_folder = "../training_data/features/original/"
dst_folder = "../training_data/model/original/"
# process(src_folder, dst_folder)
t, t1, t2 = process(src_folder, dst_folder)

NameError: name 'samples_test_size' is not defined

In [21]:
print(x.shape[0])
print(y.shape[0])

print(reader_train.streams.label)
data_x = np.reshape(x,(256,256))
data_y = np.reshape(y,(3,256,256))

NameError: name 'x' is not defined

In [22]:
print(data_y[2][0])

NameError: name 'data_y' is not defined

In [23]:
print(temp_y[0])

NameError: name 'temp_y' is not defined

In [24]:
src_folder = "../training_data/features/original/"
dst_folder = "../training_data/model/original/"
reader_train, reader_test = load_train(src_folder)
minibatch_size = 1
num_samples_per_sweep = samples_train_size
num_minibatches_to_train = (num_samples_per_sweep ) / minibatch_size


# input_map={
#         label  : cntk.input_variable((256, 256)),
#         input  : cntk.input_variable((126,3,256,256))
#     }

input_map={
        label  : reader_train.streams.label,
        input  : reader_train.streams.feature
} 

print(reader_train.streams.label)
print(reader_train.streams.feature)

label([65536])
feature([196608])


In [25]:
def reshape(x, y, index) :
    data_x = np.reshape(x,(256,256))
    data_y = np.reshape(y,(3,256,256))
    print(data_y)
    print(">>>>>>>>>>>> " + str(index))

data_x, data_y = [], []
for i in range(0, int(num_minibatches_to_train)):
    
    # Read a mini batch from the training data file
    data = reader_train.next_minibatch(minibatch_size, input_map=input_map) 
#     if i < 30:
    training_data = []
    for k in data.keys() :
        for v in data[k].as_sequences()[0] :
             training_data.append(v)

    data_x.append(training_data[0])
    data_y.append(training_data[1])
#     reshape(data_x, data_y, i+1)
#     else : 
#         break
#             data_x = np.reshape(data_x,(256,256))
#             data_y = np.reshape(data_y,(3,256,256))


In [26]:
labels = np.reshape(data_x, (180,1,256,256))
features = np.reshape(data_y,(180,3,256,256))

In [27]:
print(labels.shape)
print(features.shape)


(180, 1, 256, 256)
(180, 3, 256, 256)


In [28]:
from datetime import datetime
create_folder(dst_folder)
time = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')

path_log = dst_folder + time + ".txt"
file_writer = open(path_log, 'w')

In [ ]:

trainer, training_errors, test_errors = train(features, labels, dst_folder, path_log)

(3, 256, 256)
180
dice = Composite(Tensor[3,256,256], Tensor[1,256,256]) -> np.float32
epoch #0: training_error=-0.29813248962163924, test_error=-0.30808837711811066
epoch #1: training_error=-0.408703538775444, test_error=-0.42861929535865784


In [1]:
plot_errors({"training": training_errors, "test": test_errors}, title="Simulation Learning Curve")

NameError: name 'plot_errors' is not defined

In [ ]:
# z.save(dst_folder + "/model{0:.2f}.model".format(test_result*100 / num_minibatches_to_test))
z.save(dst_folder + time)
# test_errors[1]

In [ ]:
plot_errors({"training": training_errors, "test": test_errors}, title="Simulation Learning Curve")

In [ ]:
reader_train.stream_infos()

In [ ]:
data = reader_train.next_minibatch(minibatch_size, input_map=input_map) 

In [ ]:
data

In [ ]:
count = 0
for k in data.keys() :
    for v in data[k].as_sequences()[0] :
         print(v)

In [ ]:
from segmentation import segmentation
loss = []
loss_per_sweep = []
error = []

def process1(tissue, src_testing_data):
    s = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')
    
    error_rates = []
    sum_error_rate = 0
    
    src_root = "../training_data/features/"
    src = src_root + tissue + "/"
    
    dst_root = "../training_data/model/"
    dst = dst_root + tissue + "/"
    
    create_dir(dst_root)
    create_dir(dst)
    
    path_log = dst + s  + ".txt"
    file_writer = open(path_log, 'w')
    
    for index in range(1,2):
        src_k =  src + "k-" + str(index+1) + "/"
        filter, stride, filters,  train_per_sweep, reader_train, reader_test, num_samples_train, num_samples_test = load_config(src_k)
        
        
        string_filter_details = "folder => {:s}, train_per_sweep => {:d}\nfilter => {:d}, stride => {:d}, dept => {:d}, {:d} {:d} {:d}".format(src_k, train_per_sweep, filter, stride, len(filters), filters[0], filters[1], filters[2])
        string_append_and_print(string_filter_details, path_log)
        
        error_rate, l, ls,e = do_train_test(reader_train, reader_test, input_dim, index+1, path_log, num_output_classes, filter, stride, filters, train_per_sweep, num_samples_train, num_samples_test, dst)
        
        segmentation().segment1(dst, "model"+str(error_rate)+".model", src_testing_data)
    
# process1("granulation")

In [ ]:

plotdata = defaultdict(list)
src_testing_data = '../testing_data/granulation/'
process1("/granulation", src_testing_data)

In [ ]:
plotdata = defaultdict(list)
src_testing_data = '../testing_data/necrotic/'
process1("necrotic", src_testing_data)

In [ ]:
plotdata = defaultdict(list)
src_testing_data = '../testing_data/slough/'
process1("/slough", src_testing_data)

In [ ]:
# plotdata = defaultdict(list)
# src_testing_data = '../testing_data/original/'
# process1("/original", src_testing_data)

In [ ]:
plot(plotdata)

In [ ]:
# process1("granulation")
# plot_1000 = plotdata
plot(plot_1000)

In [ ]:
process1("augmen_motoC_to_s8_and_s8_to_iPad/necrotic")

In [ ]:
process1("augmen_motoC_to_s8_and_s8_to_iPad/slough")

In [ ]:
process1("augmen_motoC_to_s8_and_s8_to_iPad/original")

In [ ]:
loss = []
loss_per_sweep = []
error = []




        
# def process1(filter, stride, filters, train_per_sweep, path_log, reader_train, reader_test, num_samples_train, num_samples_test) :
#     error_rates = []
#     sum_error_rate = 0
#     file_writer = open(path_log, 'a')
    
#     string_filter_details = "filter => {:d}, stride => {:d}, dept => {:d}, {:d} {:d} {:d}".format(filter, stride, len(filters), filters[0], filters[1], filters[2])
#     string_append_and_print(string_filter_details, path_log)

#     file_writer.close()
#     file_writer = open(path_log, 'a')
    
#     #k-flod validation = 5
    
#     src_root = "../training_data/features/"
#     src_granulation = src_root + "granulation/"
    
#     dst_root = "../training_data/model/"
#     dst_granulation = dst_root + "granulation/"
    
    
    
#     for index in range(5):
#         #address file train and test
#         src_granulation_k = "k-" + str(index+1)
#         string_index_data_set = '\tData Set : ' + str(i)
#         string_append_and_print(string_index_data_set, path_log)
        
#         #read file train and test
# #         train_file = os.path.join(data_dir, "train.dat")
# #         test_file = os.path.join(data_dir, "test.dat")
        
#         #start train and test
#         error_rate, l, ls,e = do_train_test(reader_train, reader_test, input_dim, i, file_writer, num_output_classes, filter, stride, filters, train_per_sweep, num_samples_train, num_samples_test)
#         sum_error_rate += error_rate
#         loss.append(l)
#         loss_per_sweep.append(ls)
#         error.append(e)
#         #append error rate to list for calculate average error late
#         error_rates.append(error_rate)

#         if index_model > 1 :
#             break

#     string_file_end_line = "\tAverage model error: {0:.2f}%\n\tmin model error: {0:.2f}%\n\t>>>>>>>>>>>>>>>>>>\n>>>>>>>>>>>>>>>>>>".format(sum_error_rate/train_per_sweep,min(error_rates))     
#     string_append_and_print(string_file_end_line, path_log)
    
#     file_writer.close()

In [ ]:
# filter, stride, filters, train_per_sweep, path_log, reader_train, reader_test, num_samples_train, num_samples_test = load_config_180_test_less_filter_per_10_sweep()
# filter, stride, filters, train_per_sweep, path_log, reader_train, reader_test, num_samples_train, num_samples_test = load_config_180_test_less_filter_per_50_sweep()
# filter, stride, filters, train_per_sweep, path_log, reader_train, reader_test, num_samples_train, num_samples_test = load_config_180_train_less_filter_per_10_sweep()
# filter, stride, filters, train_per_sweep, path_log, reader_train, reader_test, num_samples_train, num_samples_test = load_config_180_train_more_filter_per_10_sweep()
# filter, stride, filters, train_per_sweep, path_log, reader_train, reader_test, num_samples_train, num_samples_test = load_config_180_train_less_filter_per_50_sweep()
# filter, stride, filters, train_per_sweep, path_log, reader_train, reader_test, num_samples_train, num_samples_test = load_config_14400_test_less_filter_per_10_sweep()
filter, stride, filters, train_per_sweep, path_log, reader_train, reader_test, num_samples_train, num_samples_test = load_config_14400_train_less_filter_per_10_sweep()
open(path_log,'w')
print(num_samples_train)
print(num_samples_test)
# process1(5, 2, [256, 512, 1024],  50, path_log)
process1(filter, stride, filters, train_per_sweep, path_log, reader_train, reader_test, num_samples_train, num_samples_test)

In [ ]:
#load model
# from cntk.ops.functions import load_model
# current_model = load_model("output/model/model4.model")
# print(z)

import matplotlib.pyplot as plt



In [ ]:
print(len(loss_per_sweep[0]))

In [ ]:
mb = list(range(0,len(loss_per_sweep[0])*60000,60000))
plt.plot(mb, loss_per_sweep[0])
plt.show()
# plt.figure(1)
# for l in range(len(loss_per_sweep)-1) :
#     sub_plot = "33" + str((int(l) + 1))
#     plt.subplot(int(sub_plot))
#     plt.plot(mb, loss_per_sweep[l])
#     plt.title("data set " + str((int(l) + 1)))

# plt.show()

In [ ]:
mb = list(range(0,len(loss[0])*10000,10000))
plt.figure(1)
for l in range(len(loss)-1) :
    sub_plot = "33" + str((int(l) + 1))
    plt.subplot(int(sub_plot))
    plt.plot(mb, loss[l])
    plt.title("data set " + str((int(l) + 1)))

plt.show()

In [ ]:
from load_data import TestingData
testing_data, labels, size_file = TestingData().load("../testing_data/input/")

In [ ]:
prediction_label = []

for i in range(0, size_file):
    prediction_label.append(z.eval(testing_data[i])[0])

In [ ]:
import numpy as np

def predict_label(index) :
    if index == 0 :
        return "1 0 0"
    elif index == 1 :
        return "0 1 0"
    else :
        return "0 0 1"

prediction_labels = []
# path_file_prediction = "output_label/prediction_label.txt"
# file = open(path_file_prediction, 'w')
# file = open(path_file_prediction, 'a')

for current_arr in prediction_label :
# #     file.write(predict_label(np.argmax(current_arr))+"\n")
    prediction_labels.append(predict_label(np.argmax(current_arr)))
# file.close()

size_labels = len(labels)
t_bounary, t_inner, t_outer = 0, 0, 0
n_bounary_inner, n_bounary_outer = 0, 0
n_inner_bounary, n_inner_outer = 0, 0
n_outer_bounary, n_outer_inner = 0, 0,
for i in range(0, size_labels) :
    if labels[i] == "0 0 1":
        if labels[i] == prediction_labels[i]:
            t_bounary += 1 
        elif prediction_labels[i] == "0 1 0" :
            n_bounary_inner += 1
        elif prediction_labels[i] == "1 0 0" :
            n_bounary_outer += 1
    elif labels[i] == "0 1 0":
        if labels[i] == prediction_labels[i]:
            t_inner += 1 
        elif prediction_labels[i] == "0 0 1" :
            n_inner_bounary += 1
        elif prediction_labels[i] == "1 0 0" :
            n_inner_outer += 1
    elif labels[i] == "1 0 0":
        if labels[i] == prediction_labels[i]:
            t_outer += 1 
        elif prediction_labels[i] == "0 0 1" :
            n_outer_bounary += 1
        elif prediction_labels[i] == "0 1 0" :
            n_outer_inner += 1
print(str(t_bounary) + " " + str(n_bounary_inner) + " " + str(n_bounary_outer))
print(str(n_inner_bounary) + " " + str(t_inner) + " " + str(n_inner_outer))
print(str(n_outer_bounary) + " " + str( n_outer_inner) + " " + str(n_bounary_outer))

In [ ]:
#transform testing data for prediction

# def number_of_line(path):
#     file = open(path, 'r')
#     f = []
#     while True :
#         line = file.readline();
#         if not line :
#             break
#         f.append(line)
#     return len(f)

# def prediction_file(path_directory):
#     arr = []
    
#     path_file = "testing_features_data.txt"
#     size_file = number_of_line(path_directory + path_file)

#     testing_file = os.path.join(path_directory, path_file)
#     reader_test = create_reader(testing_file, False, input_dim, num_output_classes)
#     test_input_map = {
#             input  : reader_test.streams.features,
#             label : reader_test.streams.labels
#         }
#     data = reader_test.next_minibatch(size_file, input_map=test_input_map)
#     data_asarray =  data[input].asarray()

#     patch = np.reshape(data_asarray[0], (3,31,31))
#     print(z.eval(patch)[0])
# #     z.eval(data_asarray)
# #     for i in range(0, size_file):
# #         patch = np.reshape(data_asarray[i], (3,31,31))
# #         print(current_model.eval(patch)[0])
# #         arr.append(current_model.eval(patch)[0])
# #     print(len(arr))
#     return arr

# arr = prediction_file("../testing_data/input/")

In [ ]:
# #prediction and write answer in file 

# predict_labels = []

# #prediction method => index : position label max
# def predict_label(index) :
#     if index == 0 :
#         return "1 0 0"
#     elif index == 1 :
#         return "0 1 0"
#     else :
#         return "0 0 1"

# path_file_prediction = "output_label/prediction_label.txt"
# file = open(path_file_prediction, 'w')
# file = open(path_file_prediction, 'a')

# for current_arr in arr :
#     file.write(predict_label(np.argmax(current_arr))+"\n")
#     predict_labels.append(predict_label(np.argmax(current_arr)))
# file.close()



In [ ]:
# #test set data
# labels = []
# def get_label_from_testing_data(path):    
#     file_test = open(path)
#     while True :
#         line = file_test.readline()
#         if not line :
#             break
#         lines = line.split(" ")
#         current_label = lines[1] + " " + lines[2] + " " + lines[3]
#         labels.append(current_label)

# get_label_from_testing_data("output_label/t/traning_data_features_t1.txt")
# get_label_from_testing_data("output_label/t/traning_data_features_t2.txt")
# get_label_from_testing_data("output_label/t/traning_data_features_t4.txt")
# get_label_from_testing_data("output_label/t/traning_data_features_t5.txt")
# get_label_from_testing_data("output_label/t/traning_data_features_t6.txt")
# get_label_from_testing_data("output_label/t/traning_data_features_t7.txt")
# get_label_from_testing_data("output_label/t/traning_data_features_t8.txt")
# get_label_from_testing_data("output_label/t/traning_data_features_t9.txt")
# get_label_from_testing_data("output_label/t/traning_data_features_t10.txt")


In [ ]:
# print(len(labels))

In [ ]:
# size_labels = len(labels)
# t_bounary, t_inner, t_outer = 0, 0, 0
# n_bounary_inner, n_bounary_outer = 0, 0
# n_inner_bounary, n_inner_outer = 0, 0
# n_outer_bounary, n_outer_inner = 0, 0,
# for i in range(0, size_labels) :
#     if labels[i] == "0 0 1":
#         if labels[i] == predict_labels[i]:
#             t_bounary += 1 
#         elif predict_labels[i] == "0 1 0" :
#             n_bounary_inner += 1
#         elif predict_labels[i] == "1 0 0" :
#             n_bounary_outer += 1
#     elif labels[i] == "0 1 0":
#         if labels[i] == predict_labels[i]:
#             t_inner += 1 
#         elif predict_labels[i] == "0 0 1" :
#             n_inner_bounary += 1
#         elif predict_labels[i] == "1 0 0" :
#             n_inner_outer += 1
#     elif labels[i] == "1 0 0":
#         if labels[i] == predict_labels[i]:
#             t_outer += 1 
#         elif predict_labels[i] == "0 0 1" :
#             n_outer_bounary += 1
#         elif predict_labels[i] == "0 1 0" :
#             n_outer_inner += 1
# print(str(t_bounary) + " " + str(n_bounary_inner) + " " + str(n_bounary_outer))
# print(str(n_inner_bounary) + " " + str(t_inner) + " " + str(n_inner_outer))
# print(str(n_outer_bounary) + " " + str( n_outer_inner) + " " + str(n_bounary_outer))
    

In [ ]:
from segmentation import segmentation
loss = []
loss_per_sweep = []
error = []
def load_config2(src) :
    filter, stride, train_per_sweep = [5, 2, 500]
    filters = [16, 32, 64]
    reader_train, num_samples_train = load_train(src)
    reader_test, num_samples_test = load_test(src)
    print("train samples =>" + str(num_samples_train), "test samples =>" + str(num_samples_test))
    
    return filter, stride, filters,  train_per_sweep, reader_train, reader_test, num_samples_train, num_samples_test
def process3(tissue, src_testing_data):
    s = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')
    
    error_rates = []
    sum_error_rate = 0
    
    src_root = "../training_data/features/"
    src = src_root + tissue + "/"
    
    dst_root = "../training_data/model/"
    dst = dst_root + tissue + "/"
    
    create_dir(dst_root)
    create_dir(dst)
    
    path_log = dst + s  + ".txt"
    file_writer = open(path_log, 'w')
    
    for index in range(1,2):
        src_k =  src + "k-" + str(index+1) + "/"
        filter, stride, filters,  train_per_sweep, reader_train, reader_test, num_samples_train, num_samples_test = load_config2(src_k)
        
        
        string_filter_details = "folder => {:s}, train_per_sweep => {:d}\nfilter => {:d}, stride => {:d}, dept => {:d}, {:d} {:d} {:d}".format(src_k, train_per_sweep, filter, stride, len(filters), filters[0], filters[1], filters[2])
        string_append_and_print(string_filter_details, path_log)
        
        error_rate, l, ls,e = do_train_test(reader_train, reader_test, input_dim, index+1, path_log, num_output_classes, filter, stride, filters, train_per_sweep, num_samples_train, num_samples_test, dst)
        
        segmentation().segment1(dst, "model"+str(error_rate)+".model", src_testing_data)
    
# process1("granulation")
src_testing_data = '../testing_data/granulation/'
process3("augmen_motoC_to_s8_and_s8_to_iPad/granulation", src_testing_data)

In [ ]:
from segmentation import segmentation
loss = []
loss_per_sweep = []
error = []
def load_config3(src) :
    filter, stride, train_per_sweep = [5, 2, 1000]
    filters = [16, 32, 64]
    reader_train, num_samples_train = load_train(src)
    reader_test, num_samples_test = load_test(src)
    print("train samples =>" + str(num_samples_train), "test samples =>" + str(num_samples_test))
    
    return filter, stride, filters,  train_per_sweep, reader_train, reader_test, num_samples_train, num_samples_test
def process4(tissue, src_testing_data):
    s = datetime.now(timezone('Asia/Bangkok')).strftime('log_date_%Y-%m-%d_time_%H-%M-%S')
    
    error_rates = []
    sum_error_rate = 0
    
    src_root = "../training_data/features/"
    src = src_root + tissue + "/"
    
    dst_root = "../training_data/model/"
    dst = dst_root + tissue + "/"
    
    create_dir(dst_root)
    create_dir(dst)
    
    path_log = dst + s  + ".txt"
    file_writer = open(path_log, 'w')
    
    for index in range(1,2):
        src_k =  src + "k-" + str(index+1) + "/"
        filter, stride, filters,  train_per_sweep, reader_train, reader_test, num_samples_train, num_samples_test = load_config3(src_k)
        
        
        string_filter_details = "folder => {:s}, train_per_sweep => {:d}\nfilter => {:d}, stride => {:d}, dept => {:d}, {:d} {:d} {:d}".format(src_k, train_per_sweep, filter, stride, len(filters), filters[0], filters[1], filters[2])
        string_append_and_print(string_filter_details, path_log)
        
        error_rate, l, ls,e = do_train_test(reader_train, reader_test, input_dim, index+1, path_log, num_output_classes, filter, stride, filters, train_per_sweep, num_samples_train, num_samples_test, dst)
        
        segmentation().segment1(dst, "model"+str(error_rate)+".model", src_testing_data)
    
# process1("granulation")
src_testing_data = '../testing_data/granulation/'
process4("augmen_motoC_to_s8_and_s8_to_iPad/granulation", src_testing_data)